In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#產生2維都是0-25的矩陣
int_array1 = np.random.randint(0, 25, (150, 2))

#產生2維都是26-50的矩陣
int_array2 = np.random.randint(26, 50, (150, 2))

#產生2維都是51-75的矩陣
int_array3 = np.random.randint(51, 75, (150, 2))

#產生2維都是76-100的矩陣
int_array4 = np.random.randint(76, 100, (150, 2))

In [3]:
#將前面的矩陣串接在一起組成訓練及測試資料集
#訓練及測試資料集數量皆為200
data_train = np.zeros((400, 2))
data_test = np.zeros((200, 2))
#train_label = np.zeros((400, 1))
#test_label = np.zeros((200, 1))

train_label = []
test_label = []
for i in range(4):
    if i==0:
        for j in range(100):
            data_train[j] = int_array1[j]
            train_label.append([1, 0, 0, 0])
        for j in range(50):
            data_test[j] = int_array1[j+100]
            test_label.append([1, 0, 0, 0])
    if i==1:
        for j in range(100):
            data_train[j+100] = int_array2[j]
            train_label.append([0, 1, 0, 0])
        for j in range(50):
            data_test[j+50] = int_array2[j+100]
            test_label.append([0, 1, 0, 0])
    if i==2:
        for j in range(100):
            data_train[j+200] = int_array3[j]
            train_label.append([0, 0, 1, 0])
        for j in range(50):
            data_test[j+100] = int_array3[j+100]
            test_label.append([0, 0, 1, 0])
    if i==3:
        for j in range(100):
            data_train[j+300] = int_array4[j]
            train_label.append([0, 0, 0, 1])
        for j in range(50):
            data_test[j+150] = int_array4[j+100]
            test_label.append([0, 0, 0, 1])

In [4]:
#定義zscore標準化函式
def zscore(x, axis=None):
    xmean = x.mean(axis=axis, keepdims=True)
    xstd = np.std(x, axis=axis, keepdims=True)
    zscore = (x-xmean)/xstd
    return zscore

In [5]:
#資料標準化
data_train = (data_train)/100
data_test = (data_test)/100
data_train = zscore(data_train, axis=0)
data_test = zscore(data_test, axis=0)

In [6]:
#label編碼
"""
label_train_encode = np.zeros((400, 1))
label_test_encode = np.zeros((200, 1))
for i in range(4):
    if  i ==0:
        for j in range(100):
            label_train_encode[i] = 0.25
        for j in range(50):
            label_test_encode[i] = 0.25
    if  i ==1:
        for j in range(100):
            label_train_encode[i+100] = 0.5
        for j in range(50):
            label_test_encode[i+50] = 0.5
    if  i ==2:
        for j in range(100):
            label_train_encode[i+200] = 0.75
        for j in range(50):
            label_test_encode[i+100] = 0.75
    if  i ==3:
        for j in range(100):
            label_train_encode[i+300] = 0.99
        for j in range(50):
            label_test_encode[i+150] = 0.99
"""
label_train_encode = train_label
label_test_encode = test_label

In [7]:
t_ = 10 #訓練回數
d_ = 400 #樣本數
n = 25 #神經元數量
m = 2 #輸入的資料維度  
p = 4 #輸出值維度

In [8]:
#定義參數
weight_v = np.random.random(size = [m , n])
weight_w = np.random.random(size = [n , p])

theta_hj = np.zeros((n , 1))
net_hj = np.zeros_like(theta_hj)
Hj = np.zeros_like(theta_hj) 
theta_k = np.zeros((p, 1))
net_k =  np.zeros((p, 1))
Yk = np.zeros_like(train_label)
delta_k = np.zeros_like(net_k)
delta_hj = np.zeros_like(Hj)
eta = 10

In [9]:
for t in range(t_): #T iterations
    randp = np.random.permutation(len(data_train))
    for d in range(d_): #D input samples
        xi = data_train[randp[d]]
        Tk = train_label[randp[d]]
        #forward
        #compute hidden layer
        for j in range(n):
            #net_hj <- -theta_hj
            net_hj[j] = net_hj[j] - theta_hj[j]
            for i in range(m):
                #net_hj = net_hj + vij * xi
                net_hj[j] += (weight_v[i][j] * xi[i])
            #Hj = 1/(1 + exp^-net_hj)
            Hj[j] = 1.0 / (1.0 + np.exp(-net_hj[j]))
            
        #compute output layer
        for k in range(p):
            #neth_k <- -theta_hk
            net_k[k] = net_k[k] - theta_k[k]
            for j in range(n):
                #neth_k = neth_k + wjk * Hj
                net_k[k] += (weight_w[j][k] * Hj[j])
            #Yk = 1/(1 + exp^-net_k)
            Yk[d][k] = 1.0 / (1.0 + np.exp(-net_k[k]))
        
            
        #backward
        #compute delta for outputlayer
        for k in range(p):
            #delta_k = Yk * (1 - Yk) * (Tk - Yk)
            delta_k[k] = Yk[d][k] * (1.0 - Yk[d][k]) * (Tk[k] - Yk[d][k])
        #compute delta for hidden layer
        for j in range(n):
            #delta_hj = 0
            delta_hj[j] = 0
            for k in range(p):
                #delta_hj = delta_hj + wjk * delta_k
                delta_hj[j] += (weight_w[j][k] * delta_k[k]) #- (eta * delta_k[k])
            #delta_hj = Hj * (1 - Hj) * delta_hj
            delta_hj[j] = Hj[j] * (1.0 - Hj[j]) * delta_hj[j]
            
        #adjust parameters
        for k in range(p):
            #theta_k = theta_k - eta * delta_k
            theta_k[k] = theta_k[k] - (eta * delta_k[k])
            for j in range(n):
                #wjk = wjk + eta * delta_k * Hj
                weight_w[j][k] = weight_w[j][k] + (eta * delta_k[k] * Hj[j])
            for j in range(n):
                #theta_hj = theta_hj - eta * delta_hj
                theta_hj[j] = theta_hj[j] - (eta * delta_hj[j])
                for i in range(m):
                    #vij = vij + eta * delta_hj * xi
                    weight_v[i][j] = weight_v[i][j] + (eta * delta_hj[j] * xi[i])
    #print(Yk)
    print('epoch: %d'%(t))

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9


In [10]:
#recall
d_test = 200 #test樣本數
for d in range(d_test): #D input samples
        xi = data_test[d]
        Tk = label_test_encode[d]
        #forward
        #compute hidden layer
        for j in range(n):
            #net_hj <- -theta_hj
            net_hj[j] = net_hj[j] - theta_hj[j]
            weight_v_j = 0
            for i in range(m):
                #net_hj = net_hj + vij * xi
                weight_v_j += weight_v[i][j] * xi[i]
            net_hj[j] = net_hj[j] + weight_v_j
            #Hj = 1/(1 + exp^-net_hj)
            Hj[j] = 1/(1 + np.exp(-net_hj[j]))
            
        #compute output layer
        for k in range(p):
            #neth_k <- -theta_hk
            net_k[k] = net_k[k] - theta_k[k]
            for j in range(n):
                #neth_k = neth_k + wjk * Hj
                net_k[k] += weight_w[j][k] * Hj[j]
            #Yk = 1/(1 + exp^-net_k)
            Yk[d][k] = 1.0 / (1.0 + np.exp(-net_k[k]))
            
            if Yk[d][k] >= 0.99:
                Yk[d][k] = 1
            else:
                Yk[d][k] = 0
        #result = np.where(np.argmax(label_test_encode, axis=0)==np.argmax(Yk[d], axis=0), 1, 0)
        if d%4==0:
            print('YK['+ str(d) +']=')
            print(Yk[d])

YK[0]=
[1 1 1 1]
YK[4]=
[1 1 1 1]
YK[8]=
[1 1 1 1]
YK[12]=
[1 1 1 1]
YK[16]=
[1 1 1 1]
YK[20]=
[1 1 1 1]
YK[24]=
[1 1 1 1]
YK[28]=
[1 1 1 1]
YK[32]=
[1 1 1 1]
YK[36]=
[1 1 1 1]
YK[40]=
[1 1 1 1]
YK[44]=
[1 1 1 1]
YK[48]=
[1 1 1 1]
YK[52]=
[1 1 1 1]
YK[56]=
[1 1 1 1]
YK[60]=
[1 1 1 1]
YK[64]=
[1 1 1 1]
YK[68]=
[1 1 1 1]
YK[72]=
[1 1 1 1]
YK[76]=
[1 1 1 1]
YK[80]=
[1 1 1 1]
YK[84]=
[1 1 1 1]
YK[88]=
[1 1 1 1]
YK[92]=
[1 1 1 1]
YK[96]=
[1 1 1 1]
YK[100]=
[1 1 1 1]
YK[104]=
[1 1 1 1]
YK[108]=
[1 1 1 1]
YK[112]=
[1 1 1 1]
YK[116]=
[1 1 1 1]
YK[120]=
[1 1 1 1]
YK[124]=
[1 1 1 1]
YK[128]=
[1 1 1 1]
YK[132]=
[1 1 1 1]
YK[136]=
[1 1 1 1]
YK[140]=
[1 1 1 1]
YK[144]=
[1 1 1 1]
YK[148]=
[1 1 1 1]
YK[152]=
[1 1 1 1]
YK[156]=
[1 1 1 1]
YK[160]=
[1 1 1 1]
YK[164]=
[1 1 1 1]
YK[168]=
[1 1 1 1]
YK[172]=
[1 1 1 1]
YK[176]=
[1 1 1 1]
YK[180]=
[1 1 1 1]
YK[184]=
[1 1 1 1]
YK[188]=
[1 1 1 1]
YK[192]=
[1 1 1 1]
YK[196]=
[1 1 1 1]
